## H2O Algorithms - Generalized Linear Model (GLM) Example
References
* http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/glm.html#generalized-linear-model-glm
* http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html#h2ogeneralizedlinearestimator
* https://github.com/h2oai/h2o-2/wiki/Hacking-Airline-DataSet-with-H2O 

In [1]:
import h2o
import numpy as np

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_151"; OpenJDK Runtime Environment (build 1.8.0_151-8u151-b12-0ubuntu0.16.04.2-b12); OpenJDK 64-Bit Server VM (build 25.151-b12, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmplpppforf
  JVM stdout: /tmp/tmplpppforf/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmplpppforf/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster version:,3.13.0.369
H2O cluster version age:,4 months and 19 days !!!
H2O cluster name:,H2O_from_python_unknownUser_g9xmny
H2O cluster total nodes:,1
H2O cluster free memory:,6.948 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [3]:
airline_data = "datasets/allyears2k.csv"
airline_df = h2o.import_file(airline_data)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
train_075, valid_025 = airline_df.split_frame(ratios=[0.75], destination_frames=['train_075', 'valid_025'], seed=12345)

In [9]:
train_075.describe()

Rows:32944
Cols:31




,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,,1.0,,16.0,17.0,14.0,-63.0,-16.0,,,11.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,,
mean,1997.4753217581354,1.4076007770762509,14.610399465760077,3.820240407965032,1348.0649346608589,1315.543133802817,1507.185343482873,1487.7556155900927,,817.0415250121418,,125.0900355649841,125.25271755632478,114.5341042728342,9.323890934048793,9.96664592408214,,,732.1626211380137,5.404549808429119,14.175318456086583,0.024405050995628945,,0.0026104905293831957,3.97542334781957,0.25520755282481644,4.929716769069379,0.017233628053349315,7.330436085718564,,
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,,3949.0,,475.0,437.0,402.0,470.0,473.0,,,3365.0,88.0,254.0,1.0,,1.0,285.0,135.0,307.0,12.0,343.0,,
sigma,6.344585860507347,1.871463308001557,9.159599498543214,1.9031546184225476,465.9472738966881,476.95476744330216,485.4247016596124,493.71192695771407,,777.9684082023755,,74.40305017730039,73.74507897065912,70.00535449262698,29.59850691383757,26.166247798699132,,,581.6186963032113,4.215436066337475,9.848799838085414,0.15430543486289774,,0.05102700171834807,15.762332774902038,3.850363228714463,18.771758107666475,0.3857896486346992,22.487107912020416,,
zeros,0,0,0,0,0,423,0,423,,0,,0,0,0,1153,4835,,,0,472,420,32140,,32858,5499,6607,5513,6657,5341,,
missing,0,0,0,0,804,0,890,0,0,0,23,890,10,12536,890,804,0,0,27,12064,12062,0,7299,0,26271,26271,26271,26271,26271,0,0
0,1987.0,10.0,14.0,3.0,741.0,730.0,912.0,849.0,PS,1451.0,NA,91.0,79.0,nan,23.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES
1,1987.0,10.0,15.0,4.0,729.0,730.0,903.0,849.0,PS,1451.0,NA,94.0,79.0,nan,14.0,-1.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,NO
2,1987.0,10.0,17.0,6.0,741.0,730.0,918.0,849.0,PS,1451.0,NA,97.0,79.0,nan,29.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES


In [10]:
valid_025.describe()

Rows:11034
Cols:31




,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,8.0,0.0,2.0,0.0,,1.0,,16.0,17.0,15.0,-48.0,-16.0,,,11.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,,
mean,1997.573681348559,1.4135399673735727,14.573228203733914,3.821732825811129,1339.2157738095239,1306.2952691680257,1497.0121166930744,1477.9251404748957,,824.2215878194671,,123.99142510951624,124.33142960746986,113.67331310504262,9.296858980333676,10.129185267857142,,,724.2698167966622,5.312924208144795,14.148897058823529,0.0255573681348559,,0.0020844661953960484,4.261504424778761,0.3902654867256637,4.634513274336284,0.016371681415929203,8.475221238938055,,
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,,3947.0,,440.0,437.0,355.0,475.0,458.0,,,3365.0,128.0,177.0,1.0,,1.0,369.0,201.0,323.0,14.0,373.0,,
sigma,6.343405429784961,1.884453830760461,9.224322706555153,1.9106372289201605,463.4818979867449,474.0981045954784,481.05676286692284,489.81688443159146,,775.728199664727,,72.67608102198045,72.36531197024428,68.53758552934492,30.5523741712606,27.238173697539914,,,568.8216643481054,4.161539745957965,10.070127542937193,0.15781776298733638,,0.04561041254356577,17.451234158484382,5.773494481736044,18.165903440046552,0.4534039511182074,26.206484303726793,,
zeros,0,0,0,0,0,146,0,146,,0,,0,0,0,361,1558,,,0,151,137,10752,,11011,1845,2233,1875,2257,1799,,
missing,0,0,0,0,282,0,305,0,0,0,9,305,3,4113,305,282,0,0,8,3962,3962,0,2475,0,8774,8774,8774,8774,8774,0,0
0,1987.0,10.0,18.0,7.0,729.0,730.0,847.0,849.0,PS,1451.0,NA,78.0,79.0,nan,-2.0,-1.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,NO,NO
1,1987.0,10.0,23.0,5.0,731.0,730.0,902.0,849.0,PS,1451.0,NA,91.0,79.0,nan,13.0,1.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES
2,1987.0,10.0,25.0,7.0,729.0,730.0,851.0,849.0,PS,1451.0,NA,82.0,79.0,nan,2.0,-1.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,NO


In [11]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [19]:
x = airline_df.col_names[:-2]
y = airline_df.col_names[-2]
airline_glm_v1 = H2OGeneralizedLinearEstimator(model_id = "glm-v1-00001",
                                               nfolds = 10,
                                               response_column = y,
                                               ignored_columns=["ActualElapsedTime","ArrDelay","DepDelay","Cancelled","Diverted","IsDepDelayed"],
                                               ignore_const_cols = True,
                                               family = "binomial",
                                               solver = "AUTO",
                                               alpha = 0.5,
                                               standardize = True,
                                               link = "logit")
                                               

In [20]:
airline_glm_v1.train(x,y,training_frame = train_075, validation_frame=valid_025)

glm Model Build progress: |███████████████████████████████████████████████| 100%
